In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import string as str


In [2]:
df = pd.read_json("scout_car.json",  lines=True)
# we have to read this file line by line

In [3]:
df.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,...,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,...,"[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,...,[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,...,"[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,NaN,...,"[\nAudi A1: , - 1e eigenaar , - Perfecte staat...",NaN,[\n6\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,NaN,...,"[\n, Technik & Sicherheit:, Xenon plus, Klimaa...",NaN,NaN,[\nGermany\n],NaN,NaN,NaN,NaN,NaN,NaN


Vat stands for value added tax which is generally 20 percent of cars.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            15919 non-null  object 
 1   make_model                     15919 non-null  object 
 2   short_description              15873 non-null  object 
 3   body_type                      15859 non-null  object 
 4   price                          15919 non-null  int64  
 5   vat                            11406 non-null  object 
 6   km                             15919 non-null  object 
 7   registration                   15919 non-null  object 
 8   prev_owner                     9091 non-null   object 
 9   kW                             0 non-null      float64
 10  hp                             15919 non-null  object 
 11  Type                           15917 non-null  object 
 12  Previous Owners                9279 non-null  

In [5]:
df = df.drop("kW", axis=1)
# kW column dropped


In [6]:
df.hp = df.hp.str.replace(r'[A-Za-z -]', "")
df.hp

<ipython-input-6-7597322c733e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.hp = df.hp.str.replace(r'[A-Za-z -]', "")


0         66
1        141
2         85
3         66
4         66
        ... 
15914    147
15915    165
15916    146
15917    147
15918    165
Name: hp, Length: 15919, dtype: object

In [7]:
df.hp = pd.to_numeric(df.hp)

In [8]:
# taking only first part of type columns as "used" etc.

df.Type = df.Type.astype("str").str.findall(r'[A-Za-z-]+').apply(lambda x : x[0])
df.Type

0                  Used
1                  Used
2                  Used
3                  Used
4                  Used
              ...      
15914               New
15915              Used
15916    Pre-registered
15917    Pre-registered
15918     Demonstration
Name: Type, Length: 15919, dtype: object

In [9]:
df.Type

0                  Used
1                  Used
2                  Used
3                  Used
4                  Used
              ...      
15914               New
15915              Used
15916    Pre-registered
15917    Pre-registered
15918     Demonstration
Name: Type, Length: 15919, dtype: object

In [10]:
df.prev_owner = df.prev_owner.str.replace('[A-Za-z]','',regex=True)
df.prev_owner

0         2  
1        None
2         1  
3         1  
4         1  
         ... 
15914    None
15915     1  
15916     1  
15917    None
15918     1  
Name: prev_owner, Length: 15919, dtype: object

In [11]:
def list_destroyer(x):
    if len(x)>0 :
        return x[0]

In [12]:
# Since some of null values refers to "0", we will use registeration dates to find non-used cars 
#and will refill some of null values.
df["First Registration"] = df["First Registration"].astype("str").str.findall(r'[0-9]+').apply(list_destroyer)
df["First Registration"]


0        2016
1        2017
2        2016
3        2016
4        2016
         ... 
15914    None
15915    2019
15916    2019
15917    2019
15918    2019
Name: First Registration, Length: 15919, dtype: object

In [13]:
count = 0
while count < len(df.prev_owner):
    for i in df["First Registration"]:
        if i == "2019":
            df.prev_owner[count]=0
        count +=1        
        

<ipython-input-13-70c81092061c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.prev_owner[count]=0


In [14]:
pd.to_numeric(df.prev_owner)

0        2.0
1        NaN
2        1.0
3        1.0
4        1.0
        ... 
15914    NaN
15915    0.0
15916    0.0
15917    0.0
15918    0.0
Name: prev_owner, Length: 15919, dtype: float64

In [15]:
df["Next Inspection"] = df["Next Inspection"].astype("str").str.findall(r'\d\d.\d\d\d\d').apply(list_destroyer)

In [16]:
pd.to_datetime(df["Next Inspection"])

0       2021-06-01
1              NaT
2              NaT
3              NaT
4              NaT
           ...    
15914          NaT
15915   2022-01-01
15916          NaT
15917          NaT
15918          NaT
Name: Next Inspection, Length: 15919, dtype: datetime64[ns]

In [17]:
df["Inspection new"] = df["Inspection new"].astype("str").str.findall(r'[A-Z][a-z]+').apply(list_destroyer)
df["Inspection new"]

0         Yes
1        None
2        None
3        None
4         Yes
         ... 
15914    None
15915    None
15916     Yes
15917    None
15918     Yes
Name: Inspection new, Length: 15919, dtype: object

In [18]:
df = df.drop(["Warranty","Full Service", "Non-smoking Vehicle"], axis = 1)

In [19]:
df = df.drop("null", axis = 1) 

In [20]:
df = df.rename(columns= {"Make":"Brand"} )
df.Brand = df.Brand.astype("str").str.findall(r'[A-Z][a-z]+').apply(list_destroyer)
df.Brand

0           Audi
1           Audi
2           Audi
3           Audi
4           Audi
          ...   
15914    Renault
15915    Renault
15916    Renault
15917    Renault
15918    Renault
Name: Brand, Length: 15919, dtype: object

In [21]:
df.Model = df.Model.astype("str").str.findall(r'[A-Z][a-z0-9]+').apply(list_destroyer)

In [22]:
df["Offer Number"] = df["Offer Number"].astype("str").apply(lambda x : x.replace("['\\n","").replace("\\n']", ""))
df["Offer Number"]

0             LR-062483
1                   nan
2              AM-95365
3                   nan
4                 C1626
              ...      
15914          10988301
15915       507370_3223
15916          Espace16
15917           2691331
15918    Re_30000008029
Name: Offer Number, Length: 15919, dtype: object

In [23]:
df["Body Color"] = df["Body Color"].astype("str").str.findall(r'[A-Z][a-z0-9]+').apply(list_destroyer)
df["Body Color"] 

0        Black
1          Red
2        Black
3        Brown
4        Black
         ...  
15914     Grey
15915     Grey
15916    White
15917     Grey
15918     Grey
Name: Body Color, Length: 15919, dtype: object

In [24]:
df["Paint Type"] = df["Paint Type"].astype("str").str.findall(r'[A-Z][a-z0-9/]+').apply(list_destroyer)
df["Paint Type"]

0        Metallic
1            None
2        Metallic
3        Metallic
4        Metallic
           ...   
15914    Metallic
15915    Metallic
15916        None
15917        None
15918    Metallic
Name: Paint Type, Length: 15919, dtype: object

In [25]:
df["Body Color Original"] = df["Body Color Original"].astype("str").apply(lambda x : x.replace("['\\n","").replace("\\n']", ""))
df["Body Color Original"]


0                 Mythosschwarz
1                           nan
2        mythosschwarz metallic
3                           nan
4        Mythosschwarz Metallic
                  ...          
15914              Grigio scuro
15915       Stahl-Grau Metallic
15916               arktis-weiß
15917                    Grigio
15918    Titanium-Grau Metallic
Name: Body Color Original, Length: 15919, dtype: object

In [26]:
def list_divider(x):
    if len(x)>0:
        if len(x)==1:
            return x[0]
        elif len(x)==3:
            return x[0]+" "+x[1]+" "+x[2]
        else:
            return x[0]+","+x[1]

In [27]:
df["Upholstery"].astype("str").str.findall(r'[A-Z][a-z]+').apply(list_divider)


0        Cloth,Black
1         Cloth,Grey
2        Cloth,Black
3               None
4        Cloth,Black
            ...     
15914           None
15915          Cloth
15916     Full,Black
15917           Part
15918     Full,Brown
Name: Upholstery, Length: 15919, dtype: object

In [28]:
df.Body = df["Body"].astype("str").str.findall(r'[A-Z][a-z ]+').apply(list_destroyer)
df.Body

0        Sedans
1        Sedans
2        Sedans
3        Sedans
4        Sedans
          ...  
15914       Van
15915       Van
15916       Van
15917       Van
15918       Van
Name: Body, Length: 15919, dtype: object

In [29]:
df["Nr. of Doors"] = df["Nr. of Doors"].astype("str").str.findall(r'\d').apply(list_destroyer)
df["Nr. of Doors"]

0        5
1        3
2        4
3        3
4        5
        ..
15914    5
15915    5
15916    5
15917    5
15918    5
Name: Nr. of Doors, Length: 15919, dtype: object

In [30]:
df["Nr. of Seats"] = df["Nr. of Doors"].astype("str").str.findall(r'\d').apply(list_destroyer)
df["Nr. of Seats"]

0        5
1        3
2        4
3        3
4        5
        ..
15914    5
15915    5
15916    5
15917    5
15918    5
Name: Nr. of Seats, Length: 15919, dtype: object

In [31]:
df["Model Code"] = df["Model Code"].astype("str").str.findall(r'[0-9/]+[A-Z]+').apply(list_destroyer)
df["Model Code"]

0        0588/BDF
1        0588/BCY
2            None
3            None
4        0588/BDF
           ...   
15914        None
15915        None
15916        None
15917        None
15918    3333/BHJ
Name: Model Code, Length: 15919, dtype: object

In [32]:
df["Gearing Type"] = df["Gearing Type"].astype("str").str.findall(r'[A-Z][a-z]+').apply(list_destroyer)
df["Gearing Type"].unique()

array(['Automatic', 'Manual', 'Semi'], dtype=object)

In [33]:
df["Displacement"] = df["Displacement"].astype("str").str.findall(r'[0-9]+').apply(list_divider).str.replace(",","")
df["Displacement"]

0        1422
1        1798
2        1598
3        1422
4        1422
         ... 
15914    1997
15915    1798
15916    1997
15917    1997
15918    1798
Name: Displacement, Length: 15919, dtype: object

In [34]:
df["Cylinders"] = df["Cylinders"].astype("str").str.findall(r'\d').apply(list_destroyer)
df["Cylinders"]

0           3
1           4
2        None
3           3
4           3
         ... 
15914       4
15915       4
15916       4
15917       4
15918       4
Name: Cylinders, Length: 15919, dtype: object

In [35]:
df["Weight"] = df["Weight"].astype("str").str.findall(r'[0-9]+').apply(list_divider).str.replace(",","")
df["Weight"]

0        1220
1        1255
2        None
3        1195
4        None
         ... 
15914    1758
15915    1708
15916    None
15917    1758
15918    1685
Name: Weight, Length: 15919, dtype: object

In [36]:
h = df["Drive chain"].astype("str").str.findall(r'[a-z]+').apply(list_destroyer)

count = 0
while count < len(h):
  for i in h:
    if i == 'nfront':
        df["Drive chain"][count]="front"
        count +=1
    elif i == 'nrear':
        df["Drive chain"][count]="front"
        count +=1
    else:
        df["Drive chain"][count]="Null"
        count +=1

<ipython-input-36-e2be8b564026>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Drive chain"][count]="front"
<ipython-input-36-e2be8b564026>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Drive chain"][count]="Null"
<ipython-input-36-e2be8b564026>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Drive chain"][count]="front"


In [37]:
df["Drive chain"]

0        front
1        front
2        front
3         Null
4        front
         ...  
15914    front
15915    front
15916    front
15917    front
15918     Null
Name: Drive chain, Length: 15919, dtype: object

In [38]:
df.Fuel = df.Fuel.astype("str").str.findall(r'[A-Z][a-z(]+').apply(list_divider)
df.Fuel

0        Diesel Particulate Filter
1                         Gasoline
2        Diesel Particulate Filter
3        Diesel Particulate Filter
4        Diesel Particulate Filter
                   ...            
15914    Diesel Particulate Filter
15915                  Super,Super
15916                       Diesel
15917                       Diesel
15918                        Super
Name: Fuel, Length: 15919, dtype: object

In [55]:
total_fuel = df.Consumption.astype("str").str.findall(r"[0-9]+?\.[0-9]+")

In [40]:
def second_element(x):
    if len(x)>1:
        return x[1]

In [53]:
def third_element(x):
    if len(x)>2:
        return x[2]

In [56]:
df["combined_fuel"] = total_fuel.apply(list_destroyer)
df["combined_fuel"]

0        3.8
1        5.6
2        3.8
3        3.8
4        4.1
        ... 
15914    5.3
15915    7.4
15916    5.3
15917    5.3
15918    6.8
Name: combined_fuel, Length: 15919, dtype: object

In [57]:
df["city_fuel"] = total_fuel.apply(second_element)
df["city_fuel"]

0        4.3
1        7.1
2        4.4
3        4.3
4        4.6
        ... 
15914    6.2
15915    9.2
15916    6.2
15917    6.2
15918    8.7
Name: city_fuel, Length: 15919, dtype: object

In [58]:
df["country_fuel"] = df.Consumption.astype("str").str.findall(r"[0-9]+?\.[0-9]+").apply(third_element)
df["country_fuel"]

0        3.5
1        4.7
2        3.4
3        3.5
4        3.8
        ... 
15914    4.7
15915    6.3
15916    4.7
15917    4.7
15918    5.7
Name: country_fuel, Length: 15919, dtype: object

In [63]:
df["CO2 Emission"] = df["CO2 Emission"].astype("str").str.findall(r"\d\d\d?").apply(list_destroyer)
df["CO2 Emission"]

0         99
1        129
2         99
3         99
4        109
        ... 
15914    139
15915    168
15916    139
15917    139
15918    153
Name: CO2 Emission, Length: 15919, dtype: object

In [67]:
df["Emission Class"] = df["Emission Class"].astype("str").str.findall(r"[A-Z][a-z 0-9]+").apply(list_destroyer)
df["Emission Class"].unique()

array(['Euro 6', 'None', 'Euro 5', 'Euro 6d', 'Euro 6c', 'Euro 4'],
      dtype=object)

In [69]:
df = df.rename(columns={"\nComfort & Convenience\n": "Comfort"})

In [ ]:
def king_of_lists(x):
    count = 0
    while count<len(x):
        return x[count]

In [71]:
df.Comfort.astype("str").str.findall(r'[A-Z][a-z]+')

0        [Air, Armrest, Automatic, Cruise, Electrical, ...
1        [Air, Automatic, Hill, Holder, Leather, Lumbar...
2        [Air, Cruise, Electrical, Hill, Holder, Leathe...
3        [Air, Armrest, Auxiliary, Electrical, Heads, H...
4        [Air, Armrest, Automatic, Electrical, Hill, Ho...
                               ...                        
15914    [Air, Automatic, Cruise, Electrically, Electri...
15915    [Air, Automatic, Cruise, Electrically, Electri...
15916    [Air, Armrest, Automatic, Cruise, Electrically...
15917    [Air, Automatic, Cruise, Electrically, Electri...
15918    [Air, Automatic, Cruise, Electrically, Electri...
Name: Comfort, Length: 15919, dtype: object

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            15919 non-null  object 
 1   make_model                     15919 non-null  object 
 2   short_description              15873 non-null  object 
 3   body_type                      15859 non-null  object 
 4   price                          15919 non-null  int64  
 5   vat                            11406 non-null  object 
 6   km                             15919 non-null  object 
 7   registration                   15919 non-null  object 
 8   prev_owner                     10365 non-null  object 
 9   hp                             15831 non-null  float64
 10  Type                           15919 non-null  object 
 11  Previous Owners                9279 non-null   object 
 12  Next Inspection                3535 non-null  

In [48]:
df.describe()

,price,hp
count,15919.000000,15831.000000
mean,18019.896727,88.367696
std,7386.169409,26.835135
min,13.000000,1.000000
25%,12850.000000,66.000000
50%,16900.000000,85.000000
75%,21900.000000,101.000000
max,74600.000000,294.000000
